In [ ]:
!pip freeze | grep scikit-learn

In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
df['duration'].describe()

count    2.918187e+06
mean     1.379372e+01
std      9.643098e+00
min      1.000000e+00
25%      7.033333e+00
50%      1.126667e+01
75%      1.766667e+01
max      6.000000e+01
Name: duration, dtype: float64

In [13]:
np.std(y_pred)

5.28140357655334

In [14]:
year = 2022
month = 2

In [15]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df_result = pd.DataFrame()

In [24]:
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred

In [25]:
df_result.head()

,ride_id,predictions
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [26]:
df_result.to_parquet('df_result', engine='pyarrow', compression=None, index=False)